In [ ]:
!unzip /content/text-to-gloss-sign-language-translation-main.zip -d /content/text-to-gloss-sign-language-translation-main

Archive:  /content/text-to-gloss-sign-language-translation-main.zip
8f6157cdff80530da3756dd7faa058d57f302703
   creating: /content/text-to-gloss-sign-language-translation-main/text-to-gloss-sign-language-translation-main/
   creating: /content/text-to-gloss-sign-language-translation-main/text-to-gloss-sign-language-translation-main/Data/
   creating: /content/text-to-gloss-sign-language-translation-main/text-to-gloss-sign-language-translation-main/Data/synthetic data/
  inflating: /content/text-to-gloss-sign-language-translation-main/text-to-gloss-sign-language-translation-main/Data/synthetic data/augmented_lemm_norm.txt  
  inflating: /content/text-to-gloss-sign-language-translation-main/text-to-gloss-sign-language-translation-main/Data/synthetic data/augmented_lemm_norm_tagged.txt  
  inflating: /content/text-to-gloss-sign-language-translation-main/text-to-gloss-sign-language-translation-main/Data/synthetic data/dgs_forward_output_itself.txt  
  inflating: /content/text-to-gloss-sign

In [ ]:
!pip install -q transformers
!pip install -q sentencepiece
!pip install -q sacremoses
!pip install -q nltk
!pip install -q sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.7/119.7 kB 2.6 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import MarianMTModel, MarianTokenizer, AdamW, MarianConfig,get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, TensorDataset
from nltk.translate.bleu_score import SmoothingFunction, corpus_bleu
from torch.nn.utils.rnn import pad_sequence
from nltk.translate.bleu_score import sentence_bleu

#Baseline and back + tag

nomal preprocessed text

In [ ]:
# Define paths to your source and target files
source_file_path_train = '/content/text-to-gloss-sign-language-translation-main/text-to-gloss-sign-language-translation-main/Data/t2g_phoenix/train/phoenix_sentences_train_lower_lemm_norm.txt'
target_file_path_train = '/content/text-to-gloss-sign-language-translation-main/text-to-gloss-sign-language-translation-main/Data/t2g_phoenix/train/phoenix_glosses_train_lower.txt'

source_file_path_valid='/content/text-to-gloss-sign-language-translation-main/text-to-gloss-sign-language-translation-main/Data/t2g_phoenix/dev/phoenix_sentences_dev_lower_lemm_norm.txt'
target_file_path_valid='/content/text-to-gloss-sign-language-translation-main/text-to-gloss-sign-language-translation-main/Data/t2g_phoenix/dev/phoenix_glosses_dev_lower.txt'

source_file_path_test='/content/text-to-gloss-sign-language-translation-main/text-to-gloss-sign-language-translation-main/Data/t2g_phoenix/test/phoenix_sentences_test_lower_lemm_norm.txt'
target_file_path_test='/content/text-to-gloss-sign-language-translation-main/text-to-gloss-sign-language-translation-main/Data/t2g_phoenix/test/phoenix_glosses_test_lower.txt'
# Initialize lists to store source and target sentences


In [ ]:
with open(source_file_path_train, 'r') as file:
    train_dataset_src = file.readlines()

with open(target_file_path_train, 'r') as file:
    train_dataset_tgt = file.readlines()

back+tag text

In [ ]:
src_backtag_train = '/content/text-to-gloss-sign-language-translation-main/text-to-gloss-sign-language-translation-main/Data/synthetic data/ph_reversed_tagged_output.txt'

In [ ]:
with open(source_file_path_train, 'r') as file:
    train_dataset_src = file.readlines()

with open(target_file_path_train, 'r') as file:
    train_dataset_tgt = file.readlines()

with open(src_backtag_train, 'r') as file:
    train_backtag_src = file.readlines()

with open(target_file_path_train, 'r') as file:
    train_backtag_tgt = file.readlines()

train_dataset_src = train_dataset_src+ train_backtag_src
train_dataset_tgt = train_dataset_tgt+ train_backtag_tgt


print(len(train_dataset_src)," ", len(train_dataset_tgt))
print(train_dataset_src[0],"--> ", train_dataset_tgt[0])
print(train_dataset_src[7096],"--> ", train_dataset_tgt[7096])

14192   14192
und nun die wettervorhersage fuer morgen donnerstag den zwoelf august
 -->  jetzt wetter morgen donnerstag zwoelf februar

<BT> und nun die wettervorhersage fuer morgen donnerstag den zwoelf februar
 -->  jetzt wetter morgen donnerstag zwoelf februar



In [ ]:
with open(source_file_path_valid, 'r') as file:
    valid_dataset_src = file.readlines()

with open(target_file_path_valid, 'r') as file:
    valid_dataset_tgt = file.readlines()

In [ ]:
print(train_dataset_src[1])
print(train_dataset_tgt[1])
print(len(train_dataset_src), len(train_dataset_tgt))
print(len(valid_dataset_src), len(valid_dataset_tgt))

mancherorts regnen es auch lang und ergiebig auch lokal ueberschwemmung sein wieder moeglich

ort regen durch regen koennen ueberschwemmung koennen

14192 14192
519 519


model

In [ ]:
# Define the model name
model_name = 'Helsinki-NLP/opus-mt-en-de'  # Replace with your desired Marian model

sentencepiece_options = "model_type=bpe,vocab_size=2000"
tokenizer = MarianTokenizer.from_pretrained(model_name, additional_special_tokens=["<s>", "</s>", "<pad>", "<unk>"], sentencepiece_options=sentencepiece_options)
tokenizer.model_max_length = 512  # Adjust as needed

# Apply additional configurations
config = MarianConfig.from_pretrained(model_name)
config.layer_norm = True  # Set layer normalization to True
config.dim_emb = 512
config.transformer_dim_ffn = 2048
config.transformer_heads = 8
config.transformer_ffn_activation = "relu"
config.enc_depth = 1
config.dec_depth = 2
config.enc_cell = "lstm"
config.enc_cell_depth = 2
config.dec_cell_base_depth = 2
config.dec_cell = "lstm"
config.sync_sgd = True
config.dropout_src = 0.1
config.dropout_trg = 0.1
config.optimizer_params = [0.9, 0.98, 1e-09]
config.clip_norm = 5
config.beam_size = 6
config.normalize = 0.6
config.exponential_smoothing = True
config.seed = 1111
config.tied_embeddings_all = True
config.transformer_dropout = 0.1
config.label_smoothing = 0.1

# Load the model with modified tokenizer
model = MarianMTModel.from_pretrained(model_name, config=config)
model.resize_token_embeddings(len(tokenizer))

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/768k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/797k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/298M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

Embedding(58102, 512)

In [ ]:
source_texts = train_dataset_src
target_texts = train_dataset_tgt

source_texts_val = valid_dataset_src
target_texts_val = valid_dataset_tgt

In [ ]:
print(source_texts[1])
print(target_texts[1])
print(len(source_texts), len(target_texts))
print(len(source_texts_val), len(target_texts_val))

mancherorts regnen es auch lang und ergiebig auch lokal ueberschwemmung sein wieder moeglich

ort regen durch regen koennen ueberschwemmung koennen

14192 14192
519 519


In [ ]:

# Tokenize source and target sentences
tokenized_data_train = tokenizer(source_texts, text_target=target_texts, return_tensors="pt", padding=True, truncation=True)
input_ids_train = tokenized_data_train['input_ids']
attention_mask_train = tokenized_data_train['attention_mask']
labels_train = tokenized_data_train['labels']

tokenized_data_valid = tokenizer(source_texts_val, text_target=target_texts_val, return_tensors="pt", padding=True, truncation=True)
input_ids_valid = tokenized_data_valid['input_ids']
attention_mask_valid = tokenized_data_valid['attention_mask']
labels_valid = tokenized_data_valid['labels']

# Create DataLoader for training and validation
train_dataset = TensorDataset(input_ids_train, attention_mask_train, labels_train)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

val_dataset = TensorDataset(input_ids_valid, attention_mask_valid, labels_valid)
val_loader = DataLoader(val_dataset, batch_size=16)


##normal preprocessing result

###epoch =10

using sacrebleu

In [ ]:
# Training the model
from sacrebleu import corpus_bleu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0003)

num_epochs = 10
warmup_steps = 16000  # Number of warmup steps
total_steps = len(train_loader) * num_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=total_steps)

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch in train_loader:
        optimizer.zero_grad()
        input_ids, attention_mask, labels = [b.to(device) for b in batch]

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()
        scheduler.step()  # Update learning rate

    average_loss = total_loss / len(train_loader)
    print(f"Epoch [{epoch + 1}/{num_epochs}] - Average Loss: {average_loss:.4f}")

    # Validation
    model.eval()
    references = []  # To store actual target sentences
    translations = []  # To store translated sentences

    with torch.no_grad():
        for val_batch in val_loader:
            val_input_ids, val_attention_mask, val_labels = [val_b.to(device) for val_b in val_batch]

            val_outputs = model.generate(input_ids=val_input_ids, attention_mask=val_attention_mask)
            decoded_outputs = tokenizer.batch_decode(val_outputs, skip_special_tokens=True)

            # Append actual target sentences and translations
            references.extend([tokenizer.decode(t, skip_special_tokens=True) for t in val_labels.cpu().numpy()])
            translations.extend(decoded_outputs)
    # Compute BLEU score
    bleu_score = corpus_bleu(translations,[references])
    print(f"Epoch [{epoch + 1}/{num_epochs}] - Validation BLEU Score: {bleu_score.score:.2f}")


Epoch [1/10] - Average Loss: 1.9136
Epoch [1/10] - Validation BLEU Score: 1.26
Epoch [2/10] - Average Loss: 0.6154
Epoch [2/10] - Validation BLEU Score: 5.85
Epoch [3/10] - Average Loss: 0.4802
Epoch [3/10] - Validation BLEU Score: 17.28
Epoch [4/10] - Average Loss: 0.4166
Epoch [4/10] - Validation BLEU Score: 22.40
Epoch [5/10] - Average Loss: 0.3761
Epoch [5/10] - Validation BLEU Score: 22.60
Epoch [6/10] - Average Loss: 0.3451
Epoch [6/10] - Validation BLEU Score: 23.96
Epoch [7/10] - Average Loss: 0.3201
Epoch [7/10] - Validation BLEU Score: 22.12
Epoch [8/10] - Average Loss: 0.3006
Epoch [8/10] - Validation BLEU Score: 22.21
Epoch [9/10] - Average Loss: 0.2787
Epoch [9/10] - Validation BLEU Score: 25.25
Epoch [10/10] - Average Loss: 0.2588
Epoch [10/10] - Validation BLEU Score: 25.74


In [ ]:
print(translations[3])
references[3]


mittwoch ix regen nordwest wehen kueste stark wehen


'mittwoch regen koennen nordwest wahrscheinlich nord stark wind'

###test dataset

In [ ]:
with open(source_file_path_test, 'r') as file:
    test_dataset_src = file.readlines()

with open(target_file_path_test, 'r') as file:
    test_dataset_tgt = file.readlines()


tokenized_data_test = tokenizer(test_dataset_src, text_target=test_dataset_tgt, return_tensors="pt", padding=True, truncation=True)
input_ids_test = tokenized_data_test['input_ids']
attention_mask_test = tokenized_data_test['attention_mask']
labels_test = tokenized_data_test['labels']

# Create DataLoader for training and validation
test_dataset = TensorDataset(input_ids_test, attention_mask_test, labels_test)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True)

model.eval()
references_test = []  # To store actual target sentences for the test set
translations_test = []  # To store translated sentences for the test set

with torch.no_grad():
    for test_batch in test_loader:
        test_input_ids, test_attention_mask, test_labels = [test_b.to(device) for test_b in test_batch]

        test_outputs = model.generate(input_ids=test_input_ids, attention_mask=test_attention_mask)
        decoded_test_outputs = tokenizer.batch_decode(test_outputs, skip_special_tokens=True)

        # Append actual target sentences and translations for the test set
        references_test.extend([tokenizer.decode(t, skip_special_tokens=True) for t in test_labels.cpu().numpy()])
        translations_test.extend(decoded_test_outputs)

# Compute BLEU score for the test dataset
bleu_score_test = corpus_bleu(translations_test,[references_test])
print(f"Test BLEU Score: {bleu_score_test.score :.2f}")

Test BLEU Score: 22.37


##back+tag result

###epoch =5

In [ ]:
# Training the model
from sacrebleu import corpus_bleu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0003)

num_epochs = 4
warmup_steps = 16000  # Number of warmup steps
total_steps = len(train_loader) * num_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=total_steps)

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch in train_loader:
        optimizer.zero_grad()
        input_ids, attention_mask, labels = [b.to(device) for b in batch]

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()
        scheduler.step()  # Update learning rate

    average_loss = total_loss / len(train_loader)
    print(f"Epoch [{epoch + 1}/{num_epochs}] - Average Loss: {average_loss:.4f}")

    # Validation
    model.eval()
    references = []  # To store actual target sentences
    translations = []  # To store translated sentences

    with torch.no_grad():
        for val_batch in val_loader:
            val_input_ids, val_attention_mask, val_labels = [val_b.to(device) for val_b in val_batch]

            val_outputs = model.generate(input_ids=val_input_ids, attention_mask=val_attention_mask)
            decoded_outputs = tokenizer.batch_decode(val_outputs, skip_special_tokens=True)

            # Append actual target sentences and translations
            references.extend([tokenizer.decode(t, skip_special_tokens=True) for t in val_labels.cpu().numpy()])
            translations.extend(decoded_outputs)
    # Compute BLEU score
    bleu_score = corpus_bleu(translations,[references])
    print(f"Epoch [{epoch + 1}/{num_epochs}] - Validation BLEU Score: {bleu_score.score:.2f}")


Epoch [1/4] - Average Loss: 1.2846
Epoch [1/4] - Validation BLEU Score: 6.17
Epoch [2/4] - Average Loss: 0.4584
Epoch [2/4] - Validation BLEU Score: 21.99
Epoch [3/4] - Average Loss: 0.3757
Epoch [3/4] - Validation BLEU Score: 22.04
Epoch [4/4] - Average Loss: 0.3314
Epoch [4/4] - Validation BLEU Score: 23.19


In [ ]:
# Training the model
from sacrebleu import corpus_bleu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0003)

num_epochs = 1
warmup_steps = 16000  # Number of warmup steps
total_steps = len(train_loader) * num_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=total_steps)

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch in train_loader:
        optimizer.zero_grad()
        input_ids, attention_mask, labels = [b.to(device) for b in batch]

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()
        scheduler.step()  # Update learning rate

    average_loss = total_loss / len(train_loader)
    print(f"Epoch [{epoch + 1}/{num_epochs}] - Average Loss: {average_loss:.4f}")

    # Validation
    model.eval()
    references = []  # To store actual target sentences
    translations = []  # To store translated sentences

    with torch.no_grad():
        for val_batch in val_loader:
            val_input_ids, val_attention_mask, val_labels = [val_b.to(device) for val_b in val_batch]

            val_outputs = model.generate(input_ids=val_input_ids, attention_mask=val_attention_mask)
            decoded_outputs = tokenizer.batch_decode(val_outputs, skip_special_tokens=True)

            # Append actual target sentences and translations
            references.extend([tokenizer.decode(t, skip_special_tokens=True) for t in val_labels.cpu().numpy()])
            translations.extend(decoded_outputs)
    # Compute BLEU score
    bleu_score = corpus_bleu(translations,[references])
    print(f"Epoch [{epoch + 1}/{num_epochs}] - Validation BLEU Score: {bleu_score.score:.2f}")


Epoch [1/1] - Average Loss: 0.2675
Epoch [1/1] - Validation BLEU Score: 26.49


In [ ]:
print(translations[3])
references[3]


mittwoch ix regen nordwest kueste stark regen


'mittwoch regen koennen nordwest wahrscheinlich nord stark wind'

###test dataset

In [ ]:
with open(source_file_path_test, 'r') as file:
    test_dataset_src = file.readlines()

with open(target_file_path_test, 'r') as file:
    test_dataset_tgt = file.readlines()


tokenized_data_test = tokenizer(test_dataset_src, text_target=test_dataset_tgt, return_tensors="pt", padding=True, truncation=True)
input_ids_test = tokenized_data_test['input_ids']
attention_mask_test = tokenized_data_test['attention_mask']
labels_test = tokenized_data_test['labels']

# Create DataLoader for training and validation
test_dataset = TensorDataset(input_ids_test, attention_mask_test, labels_test)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True)

model.eval()
references_test = []  # To store actual target sentences for the test set
translations_test = []  # To store translated sentences for the test set

with torch.no_grad():
    for test_batch in test_loader:
        test_input_ids, test_attention_mask, test_labels = [test_b.to(device) for test_b in test_batch]

        test_outputs = model.generate(input_ids=test_input_ids, attention_mask=test_attention_mask)
        decoded_test_outputs = tokenizer.batch_decode(test_outputs, skip_special_tokens=True)

        # Append actual target sentences and translations for the test set
        references_test.extend([tokenizer.decode(t, skip_special_tokens=True) for t in test_labels.cpu().numpy()])
        translations_test.extend(decoded_test_outputs)

# Compute BLEU score for the test dataset
bleu_score_test = corpus_bleu(translations_test,[references_test])
print(f"Test BLEU Score: {bleu_score_test.score :.2f}")

Test BLEU Score: 21.82


# Syntax aware transformer

## Baseline

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

source_texts_val_pos = []

for sentence in source_texts_val:
  tokens = nltk.word_tokenize(sentence)
  pos_tags = nltk.pos_tag(tokens)
  sent = ''
  for tup in pos_tags:
    sent = sent +" "+ tup[1]
    # print(tup[1])
  source_texts_val_pos.append(sent)
  # print(sent)
# source_texts_val_pos


source_texts_pos = []

for sentence in source_texts:
  tokens = nltk.word_tokenize(sentence)
  pos_tags = nltk.pos_tag(tokens)
  sent = ''
  for tup in pos_tags:
    sent = sent +" "+ tup[1]
    # print(tup[1])
  source_texts_pos.append(sent)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
# Tokenize source and target sentences
tokenized_data_train_pos = tokenizer(source_texts_pos, text_target=target_texts, return_tensors="pt", padding=True, truncation=True)
input_ids_train_pos = tokenized_data_train_pos['input_ids']
attention_mask_train_pos = tokenized_data_train_pos['attention_mask']
labels_train_pos = tokenized_data_train_pos['labels']

In [ ]:
tokenized_data_valid_pos = tokenizer(source_texts_val_pos, text_target=target_texts_val, return_tensors="pt", padding=True, truncation=True)
input_ids_valid_pos = tokenized_data_valid_pos['input_ids']
attention_mask_valid_pos = tokenized_data_valid_pos['attention_mask']
labels_valid_pos = tokenized_data_valid_pos['labels']

input_ids_train = torch.cat([input_ids_train, input_ids_train_pos], dim=1)
attention_mask_train = torch.cat([attention_mask_train, attention_mask_train_pos], dim=1)
labels_train = torch.cat([labels_train, labels_train_pos], dim=1)

input_ids_valid = torch.cat([input_ids_valid, input_ids_valid_pos], dim=1)
attention_mask_valid = torch.cat([attention_mask_valid, attention_mask_valid_pos], dim=1)
labels_valid = torch.cat([labels_valid, labels_valid_pos], dim=1)

# Create DataLoader for training and validation
train_dataset = TensorDataset(input_ids_train, attention_mask_train, labels_train)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

val_dataset = TensorDataset(input_ids_valid, attention_mask_valid, labels_valid)
val_loader = DataLoader(val_dataset, batch_size=16)

In [ ]:
from sacrebleu import corpus_bleu

# Training the model

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0003)

num_epochs = 10
warmup_steps = 16000  # Number of warmup steps
total_steps = len(train_loader) * num_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=total_steps)

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch in train_loader:
        optimizer.zero_grad()
        input_ids, attention_mask, labels = [b.to(device) for b in batch]

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()
        scheduler.step()  # Update learning rate

    average_loss = total_loss / len(train_loader)
    print(f"Epoch [{epoch + 1}/{num_epochs}] - Average Loss: {average_loss:.4f}")

    # Validation
    model.eval()
    references = []  # To store actual target sentences
    translations = []  # To store translated sentences

    with torch.no_grad():
        for val_batch in val_loader:
            val_input_ids, val_attention_mask, val_labels = [val_b.to(device) for val_b in val_batch]

            val_outputs = model.generate(input_ids=val_input_ids, attention_mask=val_attention_mask)
            decoded_outputs = tokenizer.batch_decode(val_outputs, skip_special_tokens=True)

            # Append actual target sentences and translations
            references.extend([tokenizer.decode(t, skip_special_tokens=True) for t in val_labels.cpu().numpy()])
            translations.extend(decoded_outputs)

    # Compute BLEU score
    bleu_score = corpus_bleu(translations,[references])
    print(f"Epoch [{epoch + 1}/{num_epochs}] - Validation BLEU Score: {bleu_score.score:.2f}")

Epoch [1/10] - Average Loss: 1.0729
Epoch [1/10] - Validation BLEU Score: 3.13
Epoch [2/10] - Average Loss: 0.2734
Epoch [2/10] - Validation BLEU Score: 9.49
Epoch [3/10] - Average Loss: 0.2072
Epoch [3/10] - Validation BLEU Score: 11.86
Epoch [4/10] - Average Loss: 0.1799
Epoch [4/10] - Validation BLEU Score: 13.33
Epoch [5/10] - Average Loss: 0.1609
Epoch [5/10] - Validation BLEU Score: 16.40
Epoch [6/10] - Average Loss: 0.1450
Epoch [6/10] - Validation BLEU Score: 18.02
Epoch [7/10] - Average Loss: 0.1310
Epoch [7/10] - Validation BLEU Score: 20.64
Epoch [8/10] - Average Loss: 0.1172
Epoch [8/10] - Validation BLEU Score: 22.98
Epoch [9/10] - Average Loss: 0.1035
Epoch [9/10] - Validation BLEU Score: 24.88
Epoch [10/10] - Average Loss: 0.0919
Epoch [10/10] - Validation BLEU Score: 25.01


In [ ]:
with open(source_file_path_test, 'r') as file:
    test_dataset_src = file.readlines()

with open(target_file_path_test, 'r') as file:
    test_dataset_tgt = file.readlines()

test_dataset_src_pos = []

for sentence in test_dataset_src_pos:
  tokens = nltk.word_tokenize(sentence)
  pos_tags = nltk.pos_tag(tokens)
  sent = ''
  for tup in pos_tags:
    sent = sent +" "+ tup[1]
  test_dataset_src_pos.append(sent)

tokenized_data_test = tokenizer(test_dataset_src, text_target=test_dataset_tgt, return_tensors="pt", padding=True, truncation=True)
input_ids_test = tokenized_data_test['input_ids']
attention_mask_test = tokenized_data_test['attention_mask']
labels_test = tokenized_data_test['labels']

tokenized_data_test_pos = tokenizer(test_dataset_src_pos, text_target=test_dataset_tgt, return_tensors="pt", padding=True, truncation=True)
input_ids_test_pos = tokenized_data_test_pos['input_ids']
attention_mask_test_pos = tokenized_data_test_pos['attention_mask']
labels_test_pos = tokenized_data_test_pos['labels']

input_ids_test = torch.cat([input_ids_test, input_ids_test_pos], dim=1)
attention_mask_test = torch.cat([attention_mask_test, attention_mask_test_pos], dim=1)
labels_test = torch.cat([labels_test, labels_test_pos], dim=1)

# Create DataLoader for training and validation
test_dataset = TensorDataset(input_ids_test, attention_mask_test, labels_test)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True)

model.eval()
references_test = []  # To store actual target sentences for the test set
translations_test = []  # To store translated sentences for the test set

with torch.no_grad():
    for test_batch in test_loader:
        test_input_ids, test_attention_mask, test_labels = [test_b.to(device) for test_b in test_batch]

        test_outputs = model.generate(input_ids=test_input_ids, attention_mask=test_attention_mask)
        decoded_test_outputs = tokenizer.batch_decode(test_outputs, skip_special_tokens=True)

        # Append actual target sentences and translations for the test set
        references_test.extend([tokenizer.decode(t, skip_special_tokens=True) for t in test_labels.cpu().numpy()])
        translations_test.extend(decoded_test_outputs)

# Compute BLEU score for the test dataset
bleu_score_test = corpus_bleu(translations_test,[references_test])
print(f"Test BLEU Score: {bleu_score_test.score :.2f}")

Test BLEU Score: 24.11


## BART

In [ ]:
# Define paths to your source and target files
SRC_TRAIN_FILENAME = '/content/text-to-gloss-sign-language-translation-main/text-to-gloss-sign-language-translation-main/Data/t2g_phoenix/train/phoenix_sentences_train_lower_lemm_norm.txt'
TGT_TRAIN_FILENAME = '/content/text-to-gloss-sign-language-translation-main/text-to-gloss-sign-language-translation-main/Data/t2g_phoenix/train/phoenix_glosses_train_lower.txt'

SRC_EVAL_FILENAME='/content/text-to-gloss-sign-language-translation-main/text-to-gloss-sign-language-translation-main/Data/t2g_phoenix/dev/phoenix_sentences_dev_lower_lemm_norm.txt'
TGT_EVAL_FILENAME='/content/text-to-gloss-sign-language-translation-main/text-to-gloss-sign-language-translation-main/Data/t2g_phoenix/dev/phoenix_glosses_dev_lower.txt'

GEN_SRC_FILENAME='/content/text-to-gloss-sign-language-translation-main/text-to-gloss-sign-language-translation-main/Data/t2g_phoenix/test/phoenix_sentences_test_lower_lemm_norm.txt'
GEN_TGT_FILENAME='/content/text-to-gloss-sign-language-translation-main/text-to-gloss-sign-language-translation-main/Data/t2g_phoenix/test/phoenix_glosses_test_lower.txt'
# Initialize lists to store source and target sentences

In [ ]:
TAG_MAPPER = {'amod': 0, 'conj': 1, 'nsubj': 2, 'rs': 3, 'neg': 4, 'appos': 5, 'da': 6, 'pcomp': 7, 'nmod': 8, 'xcomp': 9, 'fixed': 10, 'dative': 11, 'npadvmod': 12, 'predet': 13, 'ac': 14, 'parataxis': 15, 'csubjpass': 16, 'dm': 17, 'pm': 18, 'og': 19, 'flat': 20, 'sb': 21, 'oc': 22, 'op': 23, 'par': 24, 'prep': 25, 'nmc': 26, 'vo': 27, 'obj': 28, 'ag': 29, 'advmod': 30, 'attr': 31, 'ep': 32, 'csubj': 33, 'case': 34, 'aux': 35, 'mark': 36, 'ju': 37, 'nsubjpass': 38, 'uc': 39, 'nk': 40, 'poss': 41, 'dep': 42, 'cc': 43, 'cd': 44, 'compound': 45, 'ph': 46, 'iobj': 47, 'relcl': 48, 'det': 49, 'dobj': 50, 'expl:pass': 51, 'nummod': 52, 'cm': 53, 'expl': 54, 'ams': 55, 'quantmod': 56, 'mnr': 57, 'mo': 58, 'agent': 59, 'acl': 60, 'svp': 61, 'punct': 62, 'ng': 63, 'obl': 64, 'app': 65, 'cop': 66, 'meta': 67, 'rc': 68, 'ROOT': 69, 'cvc': 70, 'pobj': 71, 'preconj': 72, 'cj': 73, 'pd': 74, 'pg': 75, 're': 76, 'oa': 77, 'auxpass': 78, 'acomp': 79, 'advcl': 80, 'pnc': 81, 'adc': 82, 'avc': 83, 'cp': 84, 'ccomp': 85, 'intj': 86, 'prt': 87, 'sbp': 88, 'oprd': 89}

In [ ]:
!pip install sentencepiece
!pip install data_generator

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 5.7 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
import numpy as np

#%%############################################################################
'''                     USEFUL MAPPINGS FOR TOKENIZATION                    '''
###############################################################################

'''                 DICTIONARY TO MAP SYNTAX INFO TO TOKENS                 '''
SYNTAX_MAPPER = {
    'CC' :      0, #coordinating conjunction
    'CD' :      1, #cardinal digit
    'DT' :      2, #determiner
    'EX' :      3, #existential there (like: “there is” … think of it like “there exists”)
    'FW' :      4, #foreign word
    'IN' :      5, #preposition/subordinating conjunction
    'JJ' :      6, #adjective ‘big’
    'JJR':      7, #adjective, comparative ‘bigger’
    'JJS':      8, #adjective, superlative ‘biggest’
    'LS' :      9, #list marker 1)
    'MD' :      10, #modal could, will
    'NN' :      11, #noun, singular ‘desk’
    'NNS':      12, #noun plural ‘desks’
    'NNP':      13, #proper noun, singular ‘Harrison’
    'NNPS':     14, #proper noun, plural ‘Americans’
    'PDT' :     15, #predeterminer ‘all the kids’
    'POS' :     16, #possessive ending parent’s
    'PRP' :     17, #personal pronoun I, he, she
    'PRP$':     18, #possessive pronoun my, his, hers
    'RB'  :     19, #adverb very, silently,
    'RBR' :     20, #adverb, comparative better
    'RBS' :     21, #adverb, superlative best
    'RP'  :     22, #particle give up
    'TO'  :     23, # to go ‘to’ the store.
    'UH'  :     24, #interjection, errrrrrrrm
    'VB'  :     25, #verb, base form take
    'VBD' :     26, #verb, past tense took
    'VBG' :     27, #verb, gerund/present participle taking
    'VBN' :     28, #verb, past participle taken
    'VBP' :     29, #verb, sing. present, non-3d take
    'VBZ' :     30, #verb, 3rd person sing. present takes
    'WDT' :     31, #wh-determiner which
    'WP'  :     32, #wh-pronoun who, what
    'WP$' :     33, # possessive wh-pronoun whose
    'WRB' :     34, # wh-abverb where, when
    'PUNCT':    35, # Punctuations
    'OTHER':    36, # Other syntax element
    'PAD'  :    37, # Padding for syntax information
    'SYM'  :    38,
    }




import time, re, string

#%%############################################################################
'''                        DATA GENERATOR WITH OTHER                        '''
###############################################################################

class data_generatorV3(tf.keras.utils.Sequence):

    def __init__(self,

                 # DATA VARIABLES
                 tokenizer, src_data, tgt_data = None,

                 # TRAINING VARIABLES
                 batch_size=64, shuffle = True, training = True,
                 max_seq_len = None,

                 # CONTROL TOKEN VARIABLES
                 src_lan_id = 250004,
                 tgt_lan_id = 250005,
                 pad_id = 1,
                 bos_id = 3,
                 eos_id = 2,

                 # VARIABLES FOR AVANCED TAGGING
                 include_syntax = False,
                 spacy_model = "en_core_web_sm",    # "de_core_news_sm"
                 ):
        assert len(src_data) == len(tgt_data), 'Source and target must have same length'
        self.src_data = src_data
        self.tgt_data = tgt_data
        self.tokenizer = tokenizer
        self.max_seq_len = max_seq_len
        self.batch_size = batch_size
        self.training = training
        self.include_syntax = include_syntax
        self.shuffle = shuffle

        #Setting special token values
        self.src_lan_token = src_lan_id
        self.tgt_lan_token = tgt_lan_id
        self.bos_token = bos_id
        self.eos_token = eos_id
        self.pad_token = pad_id
        self.control_tokens_list = [src_lan_id, tgt_lan_id, bos_id,
                                    eos_id, pad_id, 0]

        # PRECLEANING CORPUS - Deleting empty sentences
        # self.src_data = [sen for sen in self.src_data if sen != '' and sen != ' ']
        # if self.tgt_data != None:
        #     self.tgt_data = [sen for sen in self.tgt_data if sen != '' and sen != ' ']
        self.n_sentences = len(self.src_data)
        self.len_ = int(np.ceil(self.n_sentences/batch_size))
        self.indexes = np.arange(self.n_sentences)
        self.on_epoch_end()

        # PREPROCESSING SOURCE SENTENCES
        if self.include_syntax:
            #src_pos_tags = ['    '*self.seq_length for i in range(self.n_sentences)]

            src_dep_tags = ['    '*50 for i in range(self.n_sentences)]
            src_tokens = [[1]*50 for i in range(self.n_sentences)]

            sentence_counter = 0
            import spacy
            nlp = spacy.load(spacy_model)
            to_disable = [com for com in nlp.component_names if com not in ['tok2vec', 'parser']]
            for sen in self.src_data:
                sen = self._prepare_sentence(sen)
                with nlp.select_pipes(disable=to_disable):
                    data = nlp(sen)
                words = [t.text for t in data]


                tokens = [self.tokenizer.tokenize(w) for w in words]
                token_length = [len(t) for t in tokens]
                src_tokens[sentence_counter] = [t for t_list in tokens for t in t_list]

                dep = [[TAG_MAPPER[t.dep_]]*l for t, l in zip(data, token_length)]
                src_dep_tags[sentence_counter] = [t for t_list in dep for t in t_list]
                assert len(src_tokens[sentence_counter]) == len(src_dep_tags[sentence_counter])
                sentence_counter += 1

            self.src_dep_tags = src_dep_tags
            self.src_tokens = src_tokens
        else:
            src_tokens = [[1]*50 for i in range(self.n_sentences)]
            sentence_counter = 0
            for sen in self.src_data:
                sen = self._prepare_sentence(sen)
                words = [w for w in sen.split(' ') if w != '']
                tokens = [self.tokenizer.tokenize(w) for w in words]
                src_tokens[sentence_counter] = [t for t_list in tokens for t in t_list]
                sentence_counter += 1
            self.src_tokens = src_tokens
            self.src_dep_tags = None

        # PREPROCESSING TARGET SENTENCES
        if tgt_data != None:
            tgt_tokens = [[1]*50 for i in range(self.n_sentences)]
            sentence_counter = 0
            for sen in self.tgt_data:
                sen = self._prepare_sentence(sen)
                words = [w for w in sen.split(' ') if w != '']
                tokens = [self.tokenizer.tokenize(w) for w in words]
                tgt_tokens[sentence_counter] = [t for t_list in tokens for t in t_list]
                sentence_counter += 1
            self.tgt_tokens = tgt_tokens



    def _prepare_sentence(self, sentence):
        # This function prepare the text for processing
        sentence = re.sub('([.,!?()\]\[;])', r' \1 ', sentence)
        sentence = re.sub('\s{2,}', ' ', sentence)
        patterns1 = re.findall(r'(?<=\|).+?(?=\|)',
                              re.sub('([0-9][\s*][.,][\s*][0-9])', r'|\1|', sentence))

        patterns2 = re.findall(r'(?<=\|).+?(?=\|)',
                   re.sub('([a-z][\s*][.][\s*][a-z][\s*][.])', r'|\1|', sentence))

        patterns = [patterns1[i] for i in range(0,len(patterns1),2)] + \
                    [patterns2[i] for i in range(0,len(patterns2),2)]

        for pat in patterns:
            sentence = sentence.replace(pat, pat.replace(" ", ""))

        return sentence


    def __len__(self):
        return self.len_


    def __getitem__(self, index):
        if index > self.len_-1:
            raise IndexError()

        if index == self.len_-1:
            indexes = self.indexes[index*self.batch_size:]
        else:
            indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        src_ids = [self.src_tokens[i] for i in list(indexes)]

        header = [self.bos_token, self.src_lan_token]
        src_ids = [header+ids+[self.eos_token] for ids in src_ids]

        max_length_src = max([len(ids) for ids in src_ids])
        max_length_src = max_length_src if self.max_seq_len == None else min(self.max_seq_len,max_length_src)

        att_mask = [[1]*len(ids)+[0]*(max_length_src-(len(ids))) for ids in src_ids]
        src_ids = [ids+[self.pad_token]*(max_length_src-len(ids)) for ids in src_ids]
        src_dep_tags = None

        correct_none_lamb = lambda  x : x if x != None else 0
        src_ids = [list(map(correct_none_lamb,i)) for i in src_ids]

        if self.include_syntax:
            src_dep_tags = [self.src_dep_tags[i] for i in list(indexes)]
            src_dep_tags = [[SYNTAX_MAPPER['PAD'], SYNTAX_MAPPER['PAD']]+tags+[SYNTAX_MAPPER['PAD']]
                            for tags in src_dep_tags]
            src_dep_tags = [tags+[SYNTAX_MAPPER['PAD']]*(max_length_src-(len(tags))) for tags in src_dep_tags]


        if self.training or self.tgt_data != None:
            tgt_ids = [self.tgt_tokens[i] for i in list(indexes)]
            header = [self.bos_token, self.tgt_lan_token]
            tgt_ids = [header+ids+[self.eos_token] for ids in tgt_ids]

            max_length_tgt = max([len(ids) for ids in tgt_ids]+[max_length_src])
            max_length_tgt = max_length_tgt if self.max_seq_len == None else min(self.max_seq_len,max_length_tgt)


            tgt_ids = [ids+[self.pad_token]*(max_length_tgt-len(ids))
                       for ids in tgt_ids]
            tgt_ids = [list(map(correct_none_lamb,i)) for i in tgt_ids]

            src_ids = src_ids if max_length_src > max_length_tgt \
                        else [ids+[self.pad_token]*(max_length_tgt-len(ids)) for ids in src_ids]

            att_mask = att_mask if max_length_src > max_length_tgt \
                        else [a+[0]*(max_length_tgt-max_length_src) for a in att_mask]

            outputs = [tf.convert_to_tensor(src_ids, dtype = tf.int32),
                        tf.convert_to_tensor(att_mask, dtype = tf.int32)]
            if self.include_syntax:

                src_dep_tags = src_dep_tags if max_length_src > max_length_tgt \
                        else [d+[SYNTAX_MAPPER['PAD']]*(max_length_tgt-max_length_src) for d in src_dep_tags]
                outputs = outputs + [tf.convert_to_tensor(src_dep_tags, dtype = tf.int32)]
            return outputs, tf.convert_to_tensor(tgt_ids)

        else:
            outputs = [tf.convert_to_tensor(src_ids, dtype = tf.int32),
                       tf.convert_to_tensor(att_mask, dtype = tf.int32)]
            if self.include_syntax:
                outputs = outputs + [tf.convert_to_tensor(src_dep_tags, dtype = tf.int32)]
            return outputs

    def on_epoch_end(self):
        if self.shuffle == True:
            np.random.shuffle(self.indexes)


    def detokenize(self, ids, clean = True):
        if clean:
            ids = [[int(i) for i in t if i not in self.control_tokens_list] for t in ids]
        else:
            ids = [[int(i) for i in t if i != self.pad_token] for t in ids]
        return self.tokenizer.detokenize(ids)


#%%############################################################################
'''                              SPACY TAG GLOSSARY                         '''
###############################################################################


'                              SPACY TAG GLOSSARY                         '

In [ ]:
import torch
from transformers import MarianMTModel, MarianTokenizer, AdamW, MarianConfig,get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, TensorDataset
import sentencepiece as spm

In [ ]:
# exp_settings = {'WORKING_DIR': 'T2G_syntax', 'SRC_TRAIN_FILENAME': '/content/text-to-gloss-sign-language-translation-main/text-to-gloss-sign-language-translation-main/Data/t2g_phoenix/train/phoenix_sentences_train_lower_lemm_norm.txt', 'TGT_TRAIN_FILENAME': '/content/text-to-gloss-sign-language-translation-main/text-to-gloss-sign-language-translation-main/Data/t2g_phoenix/train/phoenix_glosses_train_lower.txt', 'SRC_EVAL_FILENAME': '/content/text-to-gloss-sign-language-translation-main/text-to-gloss-sign-language-translation-main/Data/t2g_phoenix/dev/phoenix_sentences_dev_lower_lemm_norm.txt', 'TGT_EVAL_FILENAME': '/content/text-to-gloss-sign-language-translation-main/text-to-gloss-sign-language-translation-main/Data/t2g_phoenix/dev/phoenix_glosses_dev_lower.txt',
SRC_TOKEN = '<SPOK>'
TGT_TOKEN =  '<GLOSS>'
# SPACY_MODEL = 'de_core_news_sm'
BATCH_SIZE = 64
N_EPOCHS = 1000
LR = 1e-05
VOCAB_SIZE = 3000
SCALE_DOWN_FACTOR = 4
EMBEDDING_DIM = 512
N_TAG_TOKEN =  90

TRAINED_MODEL_WEIGHTS = [str(i) for i in range(5,16, 5)] # EPOCHS FOR WHICH COMPUTE THE METRICS
BATCH_SIZE = 128


GEN_SRC_FILENAME='/content/text-to-gloss-sign-language-translation-main/text-to-gloss-sign-language-translation-main/Data/t2g_phoenix/test/phoenix_sentences_test_lower_lemm_norm.txt'
GEN_TGT_FILENAME='/content/text-to-gloss-sign-language-translation-main/text-to-gloss-sign-language-translation-main/Data/t2g_phoenix/test/phoenix_glosses_test_lower.txt'
# Initialize lists to store source and target sentences

with open(GEN_SRC_FILENAME, 'rb') as f:
    src_data = f.read().decode().split('\n')
N_SAMPLES_FOR_EXPERIMENT = len(src_data)
with open(GEN_TGT_FILENAME, 'rb') as f:
    tgt_data = f.read().decode().split('\n')


sp = spm.SentencePieceProcessor(model_file='/content/test_3000.model')

eos_token = '<end>'
bos_token = '<start>'
pad_token = '<pad>'
special_tokens = [bos_token, eos_token, pad_token,
                  SRC_TOKEN, TGT_TOKEN]
special_ids = [i[1] for i in sp.tokenize(special_tokens)]


test_data_gen = data_generatorV3(sp, src_data, tgt_data = tgt_data,
                 batch_size=BATCH_SIZE, shuffle = False, training = False,
                 include_syntax = True,
                 src_lan_id = special_ids[3],
                 tgt_lan_id = special_ids[4],
                 pad_id = special_ids[2],
                 bos_id = special_ids[0],
                 eos_id = special_ids[1])


In [ ]:
from transformers.models.mbart.modeling_tf_mbart import (TFMBartEncoder, TFMBartDecoder)

In [ ]:
from transformers import AutoTokenizer, TFBertModel

In [ ]:
from transformers import TFSharedEmbeddings, MBartConfig

In [ ]:
import tensorflow as tf
import numpy as np




#%%############################################################################
'''                        BeamHypotheses (from Huggingface)                '''
###############################################################################

class BeamHypotheses(object):
    def __init__(self, num_beams, max_length, length_penalty, early_stopping):
        """
        Initialize n-best list of hypotheses.
        """
        self.max_length = max_length - 1  # ignoring bos_token
        self.length_penalty = length_penalty
        self.early_stopping = early_stopping
        self.num_beams = num_beams
        self.beams = []
        self.worst_score = 1e9

    def __len__(self):
        """
        Number of hypotheses in the list.
        """
        return len(self.beams)

    def add(self, hyp, sum_logprobs):
        """
        Add a new hypothesis to the list.
        """
        score = sum_logprobs / len(hyp) ** self.length_penalty
        if len(self) < self.num_beams or score > self.worst_score:
            self.beams.append((score, hyp))
            if len(self) > self.num_beams:
                sorted_scores = sorted([(s, idx) for idx, (s, _) in enumerate(self.beams)])
                del self.beams[sorted_scores[0][1]]
                self.worst_score = sorted_scores[1][0]
            else:
                self.worst_score = min(score, self.worst_score)

    def is_done(self, best_sum_logprobs, cur_len):
        """
        If there are enough hypotheses and that none of the hypotheses being generated can become better than the worst
        one in the heap, then we are done with this sentence.
        """

        if len(self) < self.num_beams:
            return False
        elif self.early_stopping:
            return True
        else:
            cur_score = best_sum_logprobs / cur_len ** self.length_penalty
            ret = self.worst_score >= cur_score
            return ret

#%%############################################################################
'''                      Useful Helpers   (from Huggingface)                '''
###############################################################################

def _reorder_cache(past, beam_idx):
    if len(past) == 1:
        return past

    past_key_values = past[1]

    reordered_past = ()
    for layer_past_key_values in past_key_values:
        reordered_past += (
            tuple(tf.gather(layer_past_key_value, beam_idx) for layer_past_key_value in layer_past_key_values[:2])
            + layer_past_key_values[2:],
        )
    return (past[0], reordered_past)


def set_tensor_by_indices_to_value(tensor, indices, value):
    # create value_tensor since tensor value assignment is not possible in TF
    value_tensor = tf.zeros_like(tensor) + value
    return tf.where(indices, value_tensor, tensor)

def _create_next_token_logits_penalties(input_ids, logits, repetition_penalty):
    # create logit penalties for already seen input_ids
    token_penalties = np.ones(tf.shape(logits))
    prev_input_ids = [np.unique(input_id) for input_id in input_ids.numpy()]
    for i, prev_input_id in enumerate(prev_input_ids):
        logit_penalized = logits[i].numpy()[prev_input_id]
        logit_penalties = np.zeros(logit_penalized.shape)
        # if previous logit score is < 0 then multiply repetition penalty else divide
        logit_penalties[logit_penalized < 0] = repetition_penalty
        logit_penalties[logit_penalized > 0] = 1 / repetition_penalty
        np.put(token_penalties[i], prev_input_id, logit_penalties)
    return tf.convert_to_tensor(token_penalties, dtype=tf.float32)


def _extend_tensors(tensor_list, batch_size, num_beams, input_seq_len):
    extended_list = []
    for t in tensor_list:
        t = tf.broadcast_to(tf.expand_dims(t, 1),
                            (batch_size, num_beams, input_seq_len))

        t = tf.reshape(t, (batch_size * num_beams, input_seq_len))
        extended_list.append(t)
    return extended_list

In [ ]:
with open('/content/config_architecture.dict', 'rb') as f:
            config_dict = pickle.load(f)

# UPDATING DICTIONARY VALUES
mbarConfig_dict = MBartConfig(config_dict)

mbarConfig_dict.vocab_size = VOCAB_SIZE
mbarConfig_dict.d_model = EMBEDDING_DIM
mbarConfig_dict.max_position_embeddings = EMBEDDING_DIM

mbarConfig_dict.encoder_attention_heads  //=  SCALE_DOWN_FACTOR
mbarConfig_dict.encoder_ffn_dim //= SCALE_DOWN_FACTOR
mbarConfig_dict.encoder_layers //= SCALE_DOWN_FACTOR

mbarConfig_dict.decoder_attention_heads //= SCALE_DOWN_FACTOR
mbarConfig_dict.decoder_ffn_dim //= SCALE_DOWN_FACTOR
mbarConfig_dict.decoder_layers //= SCALE_DOWN_FACTOR

word_embedding_table = TFSharedEmbeddings(mbarConfig_dict.vocab_size, mbarConfig_dict.d_model)
dep_embedding_table = TFSharedEmbeddings(N_TAG_TOKEN, mbarConfig_dict.d_model)

encoder = TFMBartEncoder(mbarConfig_dict)
decoder = TFMBartDecoder(mbarConfig_dict)

word_emb = word_embedding_table(tf.expand_dims([1], -1))*32.0
dep_emb = dep_embedding_table(tf.expand_dims([1], -1))*32.0

enc_inputs_embeds = word_emb+dep_emb
enc_hidden_states = encoder(input_ids = None,
        inputs_embeds = enc_inputs_embeds, attention_mask=tf.expand_dims([1], -1))['last_hidden_state']
dec_inputs_embeds = word_embedding_table(tf.expand_dims([1], -1))*32.0

dec_hidden_states = decoder(input_ids=None,
        inputs_embeds=dec_inputs_embeds,
        encoder_hidden_states=enc_hidden_states,
        encoder_attention_mask=tf.expand_dims([1], -1),
        past_key_values=None,
        use_cache=False,
        training=False)


#%%############################################################################
'''              GENERATE_SEQ FUNCTION (MODIFIED FROM HUGGINGFACE)          '''
###############################################################################
# from transformers.generation_utils import BeamHypotheses, _reorder_cache, _create_next_token_logits_penalties, _extend_tensors, set_tensor_by_indices_to_value
def beam_search_generation( enc_word_ids, enc_dep_ids, enc_input_att,
                            target_lan_token_id = special_ids[4],
                            pad_token_id = special_ids[2],
                            bos_token_id = special_ids[0],
                            eos_token_id = special_ids[1],

                            num_beams = 5,
                            early_stopping = True,
                            length_penalty = 1.0,
                            min_length= 0,
                            max_length = 200,
                            repetition_penalty = 1.0,
                            temperature = 1.0):

    ''' VARIABLE INITIALIZATION '''
    batch_size = enc_word_ids.shape[0]
    input_seq_len = int(tf.shape(enc_word_ids)[-1])
    vocab_size = word_embedding_table.vocab_size

    # TOKENS IDS INITIALIZATION
    target_lan_token_id = SPECIAL_TOKEN_2_IDS["es_XX"] if target_lan_token_id == None else target_lan_token_id
    eos_token_id = SPECIAL_TOKEN_2_IDS['</s>'] if eos_token_id == None else eos_token_id
    pad_token_id = SPECIAL_TOKEN_2_IDS['<pad>'] if pad_token_id == None else pad_token_id
    bos_token_id = SPECIAL_TOKEN_2_IDS['</s>']  if bos_token_id == None else bos_token_id

    ''' ENCODING INPUT TOKENS '''
    encoder_embs = (word_embedding_table(enc_word_ids)+
                                        dep_embedding_table(enc_dep_ids))*32.

    encoder_outputs = encoder(None, inputs_embeds = encoder_embs,
                              attention_mask=enc_input_att)

    [enc_word_ids, enc_dep_ids, enc_input_att] = _extend_tensors([enc_word_ids, enc_dep_ids, enc_input_att], batch_size, num_beams, input_seq_len)


    # expand batch_idx to assign correct encoder output for expanded input_ids (due to num_beams > 1 and num_return_sequences > 1)
    expanded_batch_idxs = tf.reshape(
        tf.repeat(tf.expand_dims(tf.range(batch_size), -1), repeats=num_beams , axis=1),
        shape=(-1,),
    )

    # expand encoder_outputs
    encoder_outputs = (tf.gather(encoder_outputs[0], expanded_batch_idxs, axis=0),)


    # Generating Beam Hypothesis
    generated_hyps = [
        BeamHypotheses(num_beams, max_length, length_penalty, early_stopping=early_stopping)
        for _ in range(batch_size)
    ]

    # for greedy decoding it is made sure that only tokens of the first beam are considered to avoid sampling the exact same tokens three times
    beam_scores = tf.concat([tf.zeros((batch_size, 1), dtype=tf.float32),
                             tf.ones((batch_size, num_beams - 1), dtype=tf.float32) * (-1e9)], -1)
    beam_scores = tf.reshape(beam_scores, (batch_size * num_beams,))
    # True when sentences are completely generated
    done = [False for _ in range(batch_size)]


    past = encoder_outputs
    encoder_hidden_states=encoder_outputs[0]
    encoder_attention_mask = enc_input_att
    input_ids = (tf.ones((batch_size * num_beams, 1),
                                 dtype=tf.int32)* bos_token_id)
    past_key_values = None
    cur_len = 1



    while cur_len < max_length:

        decoder_input_ids = tf.expand_dims(tf.gather(input_ids,cur_len-1,axis = -1), -1)
        decoder_token_embs = word_embedding_table(decoder_input_ids)*32.0
        outputs = decoder(None,
                inputs_embeds=decoder_token_embs,
                encoder_attention_mask=encoder_attention_mask,
                encoder_hidden_states=encoder_hidden_states, #Deshacer Tuple?
                return_dict = True,
                past_key_values = past_key_values
                )

        logits = word_embedding_table(outputs[0], mode = 'linear')
        next_token_logits = logits[:, -1, :]  # (batch_size * num_beams, vocab_size)
        past = outputs[1]


        ''' SETTING EOS AND BOS TOKENS & APPLYING PENALTIES (ADJUSTING LOGITS) '''
        if repetition_penalty != 1.0:
            next_token_logits_penalties = _create_next_token_logits_penalties(
                input_ids, next_token_logits, repetition_penalty
            )
            next_token_logits = tf.math.multiply(next_token_logits, next_token_logits_penalties)

        if temperature != 1.0:
            next_token_logits = next_token_logits / temperature

        # ADJUSTING LOGITS
        if cur_len == 1:
            vocab_range = tf.constant(range(vocab_size))
            next_token_logits = tf.where(vocab_range != target_lan_token_id, -1e8, next_token_logits)
        elif cur_len == max_length - 1:
            vocab_range = tf.constant(range(vocab_size))
            next_token_logits = tf.where(vocab_range != eos_token_id, -1e8, next_token_logits)



        #calculate log softmax score
        scores = tf.nn.log_softmax(next_token_logits, axis=-1)  # (batch_size * num_beams, vocab_size)
        # set eos token prob to zero if min_length is not reached
        if cur_len < min_length:
            # create eos_token_id boolean mask
            num_batch_hypotheses = batch_size * num_beams

            is_token_logit_eos_token = tf.convert_to_tensor(
                [True if token is eos_token_id else False for token in range(vocab_size)], dtype=tf.bool
            )
            eos_token_indices_mask = tf.broadcast_to(is_token_logit_eos_token, [num_batch_hypotheses, vocab_size])
            scores = set_tensor_by_indices_to_value(scores, eos_token_indices_mask, -float("inf"))

        # if do_sample == False
        # Add the log prob of the new beams to the log prob of the beginning of the sequence (sum of logs == log of the product)
        next_scores = scores + tf.broadcast_to(
            beam_scores[:, None], (batch_size * num_beams, vocab_size)
        )  # (batch_size * num_beams, vocab_size)

        # re-organize to group the beam together (we are keeping top hypothesis across beams)
        next_scores = tf.reshape(
            next_scores, (batch_size, num_beams * vocab_size)
        )  # (batch_size, num_beams * vocab_size)

        next_scores, next_tokens = tf.math.top_k(next_scores, k=2 * num_beams, sorted=True)

        # next batch beam content
        next_batch_beam = []

        # for each sentence create the possible Hyposthesis
        for batch_idx in range(batch_size):

            # if we are done with this sentence
            if done[batch_idx]:
                next_batch_beam.extend([(0, pad_token_id, 0)] * num_beams)  # pad the batch
                continue

            # next sentence beam content
            next_sent_beam = []

            # next tokens for this sentence
            for beam_token_rank, (beam_token_id, beam_token_score) in enumerate(
                zip(next_tokens[batch_idx], next_scores[batch_idx])):

                # get beam and token IDs
                beam_id = beam_token_id // vocab_size
                token_id = beam_token_id % vocab_size

                effective_beam_id = batch_idx * num_beams + beam_id
                # add to generated hypotheses if end of sentence or last iteration
                if (eos_token_id is not None) and (token_id.numpy() == eos_token_id):
                    # if beam_token does not belong to top num_beams tokens, it should not be added
                    is_beam_token_worse_than_top_num_beams = beam_token_rank >= num_beams
                    if is_beam_token_worse_than_top_num_beams:
                        continue
                    generated_hyps[batch_idx].add(
                        tf.identity(input_ids[effective_beam_id]), beam_token_score.numpy()
                    )
                else:
                    # add next predicted token if it is not eos_token
                    next_sent_beam.append((beam_token_score, token_id, effective_beam_id))

                # the beam for next step is full
                if len(next_sent_beam) == num_beams:
                    break
            # Check if we are done so that we can save a pad step if all(done)
            done[batch_idx] = done[batch_idx] or generated_hyps[batch_idx].is_done(
                tf.reduce_max(next_scores[batch_idx]).numpy(), cur_len
            )
            # update next beam content
            next_batch_beam.extend(next_sent_beam)

        # stop when we are done with each sentence
        if all(done):
            break

        beam_scores = tf.convert_to_tensor([x[0] for x in next_batch_beam], dtype=tf.float32)
        beam_tokens = tf.convert_to_tensor([x[1] for x in next_batch_beam], dtype=tf.int32)
        beam_idx = tf.convert_to_tensor([x[2] for x in next_batch_beam], dtype=tf.int32)

        # re-order batch and update current length
        input_ids = tf.stack([tf.identity(input_ids[x, :]) for x in beam_idx])
        input_ids = tf.concat([input_ids, tf.expand_dims(beam_tokens, 1)], axis=-1)
        # re-order internal states & Initialize next forward values
        encoder_outputs, past_key_values = _reorder_cache(past, beam_idx)
        cur_len += 1



    ''' FINILAZING GENERATION '''
    # finalize all open beam hypotheses and end to generated hypotheses
    for batch_idx in range(batch_size):
        # Add all open beam hypothesis to generated_hyps
        if done[batch_idx]:
            continue
        # test that beam scores match previously calculated scores if not eos and batch_idx not done
        if eos_token_id is not None and all(
            (token_id % vocab_size).numpy().item() != eos_token_id for token_id in next_tokens[batch_idx]
        ):
            assert tf.reduce_all(
                next_scores[batch_idx, :num_beams] == tf.reshape(beam_scores, (batch_size, num_beams))[batch_idx]
            ), "If batch_idx is not done, final next scores: {} have to equal to accumulated beam_scores: {}".format(
                next_scores[:, :num_beams][batch_idx], tf.reshape(beam_scores, (batch_size, num_beams))[batch_idx]
            )

        # need to add best num_beams hypotheses to generated hyps
        for beam_id in range(num_beams):
            effective_beam_id = batch_idx * num_beams + beam_id
            final_score = beam_scores[effective_beam_id].numpy().item()
            final_tokens = input_ids[effective_beam_id]
            generated_hyps[batch_idx].add(final_tokens, final_score)

    # select the best hypotheses
    sent_lengths_list = []
    best = []

    # retrieve best hypotheses
    for i, hypotheses in enumerate(generated_hyps):
        sorted_hyps = sorted(hypotheses.beams, key=lambda x: x[0])
        best_hyp = sorted_hyps.pop()[1]
        sent_lengths_list.append(len(best_hyp))
        best.append(best_hyp)


    sent_lengths = tf.convert_to_tensor(sent_lengths_list, dtype=tf.int32)

    # shorter batches are filled with pad_token
    if tf.reduce_min(sent_lengths).numpy() != tf.reduce_max(sent_lengths).numpy():
        sent_max_len = min(tf.reduce_max(sent_lengths).numpy() + 1, max_length)
        decoded_list = []

        # fill with hypothesis and eos_token_id if necessary
        for i, hypo in enumerate(best):
            # if sent_length is max_len do not pad
            if sent_lengths[i] == sent_max_len:
                decoded_slice = hypo
            else:
                # else pad to sent_max_len
                num_pad_tokens = sent_max_len - sent_lengths[i]
                padding = pad_token_id * tf.ones((num_pad_tokens,), dtype=tf.int32)
                decoded_slice = tf.concat([hypo, padding], axis=-1)

                # finish sentence with EOS token
                if sent_lengths[i] < max_length:
                    decoded_slice = tf.where(
                        tf.range(sent_max_len, dtype=tf.int32) == sent_lengths[i],
                        eos_token_id * tf.ones((sent_max_len,), dtype=tf.int32),
                        decoded_slice,
                    )
            # add to list
            decoded_list.append(decoded_slice)

        decoded = tf.stack(decoded_list)
    else:
        # none of the hypotheses have an eos_token
        decoded = tf.stack(best)
    return decoded



#%%############################################################################
'''                            GENERATION PROCESS                           '''
###############################################################################

for m in TRAINED_MODEL_WEIGHTS:
    # Finetuned
    with open('/content/encoder_125.weights'.format(m),'rb') as f:
        weights = pickle.load(f)
    encoder.set_weights(weights)

    with open('/content/decoder_125.weights'.format(m),'rb') as f:
        weights = pickle.load(f)
    decoder.set_weights(weights)

    with open('/content/word_embeddings_125.weights'.format(m),'rb') as f:
        weights = pickle.load(f)
    word_embedding_table.set_weights(weights)

    with open('/content/dep_embeddings_125.weights'.format(m),'rb') as f:
        weights = pickle.load(f)
    dep_embedding_table.set_weights(weights)

    generated_text = ['  '*20 for _ in range(test_data_gen.n_sentences)]
    sentence_counter = 0
    t0 = time.time()
    for i in range(len(test_data_gen)):
        tf.print('Generation Batch {}/{}'.format(i+1, len(test_data_gen)))

        (enc_word_ids,enc_input_att, enc_dep_ids), dec_input_ids  = test_data_gen[i]
        generated_tokens = beam_search_generation(enc_word_ids, enc_dep_ids, enc_input_att)
        n_gen_sentences = generated_tokens.shape[0]

        generated_text[sentence_counter:sentence_counter+n_gen_sentences] = test_data_gen.detokenize(generated_tokens, clean=True)
        sentence_counter += n_gen_sentences
    gen_time = time.time()-t0
    tf.print('Elapsed Time: {}'.format(gen_time))
    with open('/content/generated_text2.txt'.format(m), 'w', encoding = 'utf-8') as f:
        f.write('\n'.join(generated_text))


In [ ]:
import pickle

In [ ]:
GEN_SRC_FILENAME='/content/text-to-gloss-sign-language-translation-main/text-to-gloss-sign-language-translation-main/Data/t2g_phoenix/test/phoenix_sentences_test_lower_lemm_norm.txt'
GEN_TGT_FILENAME='/content/text-to-gloss-sign-language-translation-main/text-to-gloss-sign-language-translation-main/Data/t2g_phoenix/test/phoenix_glosses_test_lower.txt'

In [ ]:
import pandas as pd
import numpy as np
import os, sys
sys.path.insert(0, "./utils/")

# from rouge import rouge
# from sacreBLEU_script import compute_sacre_bleu

from nltk.translate.meteor_score import meteor_score
# from pyter import ter

REF_FILENAME = '/content/text-to-gloss-sign-language-translation-main/text-to-gloss-sign-language-translation-main/Data/t2g_phoenix/test/phoenix_test_glosses.txt'
WORKING_DIR = 'T2G_syntax' # 'T2G_no_syntax'
EVAL_EPOCHS = range(5,501,5)


#%%############################################################################
'''                         ACCOMODATING ES-EN DATA                         '''
###############################################################################

with open(REF_FILENAME, 'rb') as f:
    ref_data = f.read().decode().split('\n')

columns = ['Sacrebleu']

#%%############################################################################
'''                            COMPUTING METRICS                            '''
###############################################################################
results_top_container = np.zeros(shape = (len(EVAL_EPOCHS), len(columns)))
imodel = 0
for it in EVAL_EPOCHS:

    with open('/content/125.txt'.format(it), 'rb') as f:
        generated_text = f.read().decode()
        generated_text = [s.strip() for s in generated_text.split('\n')]

    hyp = [s for s in generated_text]
    ref = [s for s in ref_data]

    try:
      results_top_container[imodel, 0] = compute_sacre_bleu(hyp, ref, tokenize = 'char')
    except EOFError:
      results_top_container[imodel, 0] = 0

    imodel += 1

results = pd.DataFrame(results_top_container, columns = columns, index =EVAL_EPOCHS)
dataset_type = REF_FILENAME.split('_')[-1].split('.')[0]
# results.to_excel('{}/metrics_{}.xlsx'.format(WORKING_DIR, dataset_type))
results

,Sacrebleu
5,51.447696
10,51.447696
15,51.447696
20,51.447696
25,51.447696
...,...
480,51.447696
485,51.447696
490,51.447696
495,51.447696


# Paraphrasing

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from deep_translator import GoogleTranslator

def my_paraphrase(sentence):
    tokenizer = AutoTokenizer.from_pretrained("Vamsi/T5_Paraphrase_Paws")
    model = AutoModelForSeq2SeqLM.from_pretrained("Vamsi/T5_Paraphrase_Paws")
    text =  "paraphrase: " + sentence + " </s>"
    encoding = tokenizer.encode_plus(text,pad_to_max_length=True, return_tensors="pt")
    input_ids, attention_masks = encoding["input_ids"], encoding["attention_mask"]

    outputs = model.generate(
        input_ids=input_ids, attention_mask=attention_masks,
        max_length=256,
        do_sample=True,
        top_k=120,
        top_p=0.95,
        early_stopping=False,
        num_return_sequences=1
    )

    for output in outputs:
        line = tokenizer.decode(output, skip_special_tokens=True,clean_up_tokenization_spaces=True)

    return line
with open('converted_sentences.txt', 'r') as f:
  for sentence in (lines):
    translated_sentence=(GoogleTranslator(source='auto', target='en').translate(sentence))
    para=(my_paraphrase(translated_sentence))
    translated_para=(GoogleTranslator(source='auto', target='ge').translate(sentence))
    f.write(translated_para+"\n")


In [ ]:
# Define paths to your source and target files
source_file_path_train = '/content/text-to-gloss-sign-language-translation-main/text-to-gloss-sign-language-translation/Data/t2g_phoenix/train/phoenix_sentences_train_lower.txt'
target_file_path_train = '/content/text-to-gloss-sign-language-translation-main/text-to-gloss-sign-language-translation/Data/t2g_phoenix/train/phoenix_glosses_train_lower.txt'

source_file_path_valid='/content/text-to-gloss-sign-language-translation-main/text-to-gloss-sign-language-translation/Data/t2g_phoenix/dev/phoenix_sentences_dev_lower.txt'
target_file_path_valid='/content/text-to-gloss-sign-language-translation-main/text-to-gloss-sign-language-translation/Data/t2g_phoenix/dev/phoenix_glosses_dev_lower.txt'

source_file_path_test='/content/text-to-gloss-sign-language-translation-main/text-to-gloss-sign-language-translation/Data/t2g_phoenix/test/phoenix_sentences_test_lower.txt'
target_file_path_test='/content/text-to-gloss-sign-language-translation-main/text-to-gloss-sign-language-translation/Data/t2g_phoenix/test/phoenix_glosses_test_lower.txt'
# Initialize lists to store source and target sentences


In [ ]:
with open(source_file_path_train, 'r') as file:
    train_dataset_src = file.readlines()

with open(target_file_path_train, 'r') as file:
    train_dataset_tgt = file.readlines()

In [ ]:
src_backtag_train = '/content/converted_sentences.txt'

In [ ]:
with open(source_file_path_train, 'r') as file:
    train_dataset_src = file.readlines()

with open(target_file_path_train, 'r') as file:
    train_dataset_tgt = file.readlines()

with open(src_backtag_train, 'r') as file:
    train_backtag_src = file.readlines()

with open(target_file_path_train, 'r') as file:
    train_backtag_tgt = file.readlines()

train_dataset_src = train_dataset_src+ train_backtag_src
train_dataset_tgt = train_dataset_tgt+ train_backtag_tgt


print(len(train_dataset_src)," ", len(train_dataset_tgt))
print(train_dataset_src[0],"--> ", train_dataset_tgt[0])
print(train_dataset_src[7096],"--> ", train_dataset_tgt[7096])

14192   14192
und nun die wettervorhersage für morgen donnerstag den zwölften august
 -->  jetzt wetter morgen donnerstag zwoelf februar

... and now the Wettervorhersage for morgen - Donnerstag from December twelve - August.
 -->  jetzt wetter morgen donnerstag zwoelf februar



In [ ]:
with open(source_file_path_valid, 'r') as file:
    valid_dataset_src = file.readlines()

with open(target_file_path_valid, 'r') as file:
    valid_dataset_tgt = file.readlines()

In [ ]:
print(train_dataset_src[1])
print(train_dataset_tgt[1])
print(len(train_dataset_src), len(train_dataset_tgt))
print(len(valid_dataset_src), len(valid_dataset_tgt))

mancherorts regnet es auch länger und ergiebig auch lokale überschwemmungen sind wieder möglich

ort regen durch regen koennen ueberschwemmung koennen

14192 14192
519 519


model

In [ ]:
# Define the model name
model_name = 'Helsinki-NLP/opus-mt-en-de'  # Replace with your desired Marian model

sentencepiece_options = "model_type=bpe,vocab_size=2000"
tokenizer = MarianTokenizer.from_pretrained(model_name, additional_special_tokens=["<s>", "</s>", "<pad>", "<unk>"], sentencepiece_options=sentencepiece_options)
tokenizer.model_max_length = 512  # Adjust as needed

# Apply additional configurations
config = MarianConfig.from_pretrained(model_name)
config.layer_norm = True  # Set layer normalization to True
config.dim_emb = 512
config.transformer_dim_ffn = 2048
config.transformer_heads = 8
config.transformer_ffn_activation = "relu"
config.enc_depth = 1
config.dec_depth = 2
config.enc_cell = "lstm"
config.enc_cell_depth = 2
config.dec_cell_base_depth = 2
config.dec_cell = "lstm"
config.sync_sgd = True
config.dropout_src = 0.1
config.dropout_trg = 0.1
config.optimizer_params = [0.9, 0.98, 1e-09]
config.clip_norm = 5
config.beam_size = 6
config.normalize = 0.6
config.exponential_smoothing = True
config.seed = 1111
config.tied_embeddings_all = True
config.transformer_dropout = 0.1
config.label_smoothing = 0.1

# Load the model with modified tokenizer
model = MarianMTModel.from_pretrained(model_name, config=config)
model.resize_token_embeddings(len(tokenizer))

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/768k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/797k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/298M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

Embedding(58102, 512)

In [ ]:
source_texts = train_dataset_src
target_texts = train_dataset_tgt

source_texts_val = valid_dataset_src
target_texts_val = valid_dataset_tgt

In [ ]:
print(source_texts[1])
print(target_texts[1])
print(len(source_texts), len(target_texts))
print(len(source_texts_val), len(target_texts_val))

mancherorts regnet es auch länger und ergiebig auch lokale überschwemmungen sind wieder möglich

ort regen durch regen koennen ueberschwemmung koennen

14192 14192
519 519


In [ ]:

# Tokenize source and target sentences
tokenized_data_train = tokenizer(source_texts, text_target=target_texts, return_tensors="pt", padding=True, truncation=True)
input_ids_train = tokenized_data_train['input_ids']
attention_mask_train = tokenized_data_train['attention_mask']
labels_train = tokenized_data_train['labels']

tokenized_data_valid = tokenizer(source_texts_val, text_target=target_texts_val, return_tensors="pt", padding=True, truncation=True)
input_ids_valid = tokenized_data_valid['input_ids']
attention_mask_valid = tokenized_data_valid['attention_mask']
labels_valid = tokenized_data_valid['labels']

# Create DataLoader for training and validation
train_dataset = TensorDataset(input_ids_train, attention_mask_train, labels_train)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

val_dataset = TensorDataset(input_ids_valid, attention_mask_valid, labels_valid)
val_loader = DataLoader(val_dataset, batch_size=16)


In [ ]:
# Training the model
from sacrebleu import corpus_bleu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0003)

num_epochs = 4
warmup_steps = 16000  # Number of warmup steps
total_steps = len(train_loader) * num_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=total_steps)

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch in train_loader:
        optimizer.zero_grad()
        input_ids, attention_mask, labels = [b.to(device) for b in batch]

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()
        scheduler.step()  # Update learning rate

    average_loss = total_loss / len(train_loader)
    print(f"Epoch [{epoch + 1}/{num_epochs}] - Average Loss: {average_loss:.4f}")

    # Validation
    model.eval()
    references = []  # To store actual target sentences
    translations = []  # To store translated sentences

    with torch.no_grad():
        for val_batch in val_loader:
            val_input_ids, val_attention_mask, val_labels = [val_b.to(device) for val_b in val_batch]

            val_outputs = model.generate(input_ids=val_input_ids, attention_mask=val_attention_mask)
            decoded_outputs = tokenizer.batch_decode(val_outputs, skip_special_tokens=True)

            # Append actual target sentences and translations
            references.extend([tokenizer.decode(t, skip_special_tokens=True) for t in val_labels.cpu().numpy()])
            translations.extend(decoded_outputs)
    # Compute BLEU score
    bleu_score = corpus_bleu(translations,[references])
    print(f"Epoch [{epoch + 1}/{num_epochs}] - Validation BLEU Score: {bleu_score.score:.2f}")


Epoch [1/4] - Average Loss: 1.3975
Epoch [1/4] - Validation BLEU Score: 7.88
Epoch [2/4] - Average Loss: 0.4953
Epoch [2/4] - Validation BLEU Score: 19.48
Epoch [3/4] - Average Loss: 0.4042
Epoch [3/4] - Validation BLEU Score: 25.65
Epoch [4/4] - Average Loss: 0.3534
Epoch [4/4] - Validation BLEU Score: 25.85


Test dataset

In [ ]:
with open(source_file_path_test, 'r') as file:
    test_dataset_src = file.readlines()

with open(target_file_path_test, 'r') as file:
    test_dataset_tgt = file.readlines()


tokenized_data_test = tokenizer(test_dataset_src, text_target=test_dataset_tgt, return_tensors="pt", padding=True, truncation=True)
input_ids_test = tokenized_data_test['input_ids']
attention_mask_test = tokenized_data_test['attention_mask']
labels_test = tokenized_data_test['labels']

# Create DataLoader for training and validation
test_dataset = TensorDataset(input_ids_test, attention_mask_test, labels_test)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True)

model.eval()
references_test = []  # To store actual target sentences for the test set
translations_test = []  # To store translated sentences for the test set

with torch.no_grad():
    for test_batch in test_loader:
        test_input_ids, test_attention_mask, test_labels = [test_b.to(device) for test_b in test_batch]

        test_outputs = model.generate(input_ids=test_input_ids, attention_mask=test_attention_mask)
        decoded_test_outputs = tokenizer.batch_decode(test_outputs, skip_special_tokens=True)

        # Append actual target sentences and translations for the test set
        references_test.extend([tokenizer.decode(t, skip_special_tokens=True) for t in test_labels.cpu().numpy()])
        translations_test.extend(decoded_test_outputs)

# Compute BLEU score for the test dataset
bleu_score_test = corpus_bleu(translations_test,[references_test])
print(f"Test BLEU Score: {bleu_score_test.score :.2f}")

Test BLEU Score: 24.86
